In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os

import helper
import postgres

In [2]:
file_path = os.path.join("..", "..", "Data")
file_path = [
    os.path.join(file_path, file)
    for file in os.listdir(file_path)
    if file.startswith("applemobilitytrends")
]

postgres.create_tables_apple()


Creating connection to the PostgreSQL database...
Table countries_apple already exists! Skipping...
Table mobility_stats_apple already exists! Skipping...
Table countries_apple already exists! Skipping...
Table mobility_stats_apple already exists! Skipping...


In [28]:
df = pd.read_csv(filepath_or_buffer=file_path[0], low_memory=False)

dates = df[df.columns[6:]]
nulls = dates[dates.isna().all(axis=1)].index
df.drop(index=nulls, inplace=True)

countries_util = (
    df.loc[:, ("region", "geo_type", "alternative_name", "sub-region", "country")]
    .drop_duplicates().sort_values(by='region')
    .reset_index(drop=True)
)

df.drop(columns=["geo_type", "alternative_name", "country", "sub-region"], inplace=True)


In [29]:
inter_df = helper.rearrange_df(df)
inter_df = inter_df[inter_df["date"] > str(res["date"].values[0])]

res = helper.last_entry("mobility_stats_apple")

Creating connection to the PostgreSQL database...


In [30]:
query = "select * from countries_apple"
existing_countries = (
    pd.read_sql_query(sql=query, con=helper._make_connection().connect())
    .sort_values(by="region")
    .reset_index(drop=True)
)

Creating connection to the PostgreSQL database...


In [31]:
existing_countries.head()

,region,geo_type,alternative_name,sub-region,country
0,Aachen,city,None,North Rhine-Westphalia,Germany
1,Abbeville County,county,None,South Carolina,United States
2,Abruzzo Region,sub-region,None,None,Italy
3,Acadia Parish,county,None,Louisiana,United States
4,Accomack County,county,None,Virginia,United States


In [43]:
countries_util = pd.concat([countries_util, existing_countries]).drop_duplicates(
    keep=False
)

In [45]:
postgres.import_data(countries_util, "countries_apple")
postgres.import_data(inter_df, "mobility_stats_apple")


Creating connection to the PostgreSQL database...
Creating connection to the PostgreSQL database...


In [46]:
os.remove(file_path[0])
